In [1]:
import duckdb
import pandas as pd
import json
import yaml
import openai
from tqdm import tqdm
import os


In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model=PARAM['vector_embedding_model']):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [3]:
#con = duckdb.connect("drug.db")
con = duckdb.connect("")


con.sql("INSTALL duckpgq FROM community;")
con.sql("INSTALL fts;")
con.sql("INSTALL vss;")

con.load_extension("duckpgq")
con.load_extension("fts")
con.load_extension("vss")

In [4]:
con.sql("""CREATE TABLE Trials (
            PostingID     INTEGER NOT NULL PRIMARY KEY,
            Sponsor    VARCHAR,
            StudyTitle     VARCHAR,
            Disorder   VARCHAR,
            Phase   VARCHAR,
            LinkToSponsorStudyRegistry   VARCHAR,
            LinkToClinicalTrials   VARCHAR,
            drug_cui  VARCHAR[],
            drug_names   VARCHAR[],
            )""")

con.sql("""CREATE TABLE Drug (
            drug_cui  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR
            )""")

con.sql("""CREATE TABLE Disorder (
            disorder_cui  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR,
            definition  VARCHAR,
            definitionEmbedding FLOAT[1536]
            )""")

con.sql("""CREATE TABLE MOA (
            moa_id  VARCHAR NOT NULL PRIMARY KEY,
            name  VARCHAR
            )""")

con.sql("""CREATE TABLE DrugDisorder (
            drug_cui  VARCHAR NOT NULL REFERENCES Drug(drug_cui),
            disorder_cui  VARCHAR NOT NULL REFERENCES Disorder(disorder_cui)
            )""")

con.sql("""CREATE TABLE DrugMOA (
            drug_cui  VARCHAR NOT NULL REFERENCES Drug(drug_cui),
            moa_id  VARCHAR NOT NULL REFERENCES MOA(moa_id)
            )""")


con.sql("""SET hnsw_enable_experimental_persistence = true""")

con.sql("""CREATE INDEX my_hnsw_cosine_index ON Disorder USING HNSW (definitionEmbedding);""")


con.sql("""
CREATE PROPERTY GRAPH drug_graph
  VERTEX TABLES (
    Drug, Disorder, MOA
  )
EDGE TABLES (
  DrugDisorder 	SOURCE KEY (drug_cui) REFERENCES Drug (drug_cui)
                DESTINATION KEY (disorder_cui) REFERENCES Disorder (disorder_cui)
  LABEL MAY_TREAT,
  DrugMOA SOURCE KEY (drug_cui) REFERENCES Drug (drug_cui)
          DESTINATION KEY (moa_id) REFERENCES MOA (moa_id)
  LABEL HAS_MOA
);
          """)


┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [5]:
df = pd.read_csv("drugs/trials_umls.tsv", sep='\t', lineterminator='\n')

con.sql("""BEGIN TRANSACTION;""")


for index, row in df.iterrows():
  con.execute("""
                INSERT INTO Trials (
                    PostingID,
                    Sponsor,
                    StudyTitle,
                    Disorder,
                    Phase,
                    LinkToSponsorStudyRegistry,
                    LinkToClinicalTrials,
                    drug_cui,
                    drug_names
                ) 
                SELECT 
                    $1,
                    $2,
                    $3,
                    $4,
                    $5,
                    $6,
                    $7,
                    $8::VARCHAR[],
                    $9::VARCHAR[]
            """, [
                row["Posting ID"],
                row["Sponsor"],
                row["Study Title"],
                row["Medical Condition"],
                row["Phase"],
                row["Link to Sponsor Study Registry"],
                row["Link to study details on ClinicalTrials.gov (if available)"],
                row["CUI_umls"].split(";"),
                row["preferred_umls_name"].split(";")
            ])
  #query = f"""INSERT INTO Trials VALUES ({row["Posting ID"]}, '{row["Sponsor"]}', '{row["Study Title"].replace("'", "''")}', '{row["Medical Condition"].replace("'", "''")}', '{row["Phase"]}', '{row["Link to Sponsor Study Registry"]}', '{row["Link to study details on ClinicalTrials.gov (if available)"]}', {row["CUI_umls"].split(";")}, {row["preferred_umls_name"].split(";")});"""
  #print(query)
  #con.sql(query)

con.sql("""PRAGMA create_fts_index('Trials', 'PostingID', 'StudyTitle', overwrite=1);""")

con.sql("""COMMIT;""")



In [6]:
con.sql("""SELECT StudyTitle FROM Trials LIMIT 5;""")

┌──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│                                                                                                                      StudyTitle                                                                                                                      │
│                                                                                                                       varchar                                                                                                                        │
├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤
│ U0

In [7]:
db = "Trials"

query = f"""SELECT PostingID as trial_id, StudyTitle, score
FROM (
    SELECT *, fts_main_{db}.match_bm25(
        PostingID,
        'double blind & Valaciclovir',
        fields := 'StudyTitle'
    ) AS score
    FROM {db}
)
WHERE score IS NOT NULL
ORDER BY score DESC LIMIT 5;
"""

con.sql(query)

┌──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────┐
│ trial_id │                                                                                                      StudyTitle                                                                                                       │       score        │
│  int32   │                                                                                                        varchar                                                                                                        │       double       │
├──────────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼────────────────────┤


In [8]:
con.sql("""BEGIN TRANSACTION;""")

for line in open("drugs/drugs_4.json", "r"):
  value = json.loads(line)

  query = f"""INSERT INTO Drug VALUES ('{value["cui"]}', '{value["name"].replace("'", "''")}');"""
  #print(query)
  con.sql(query)

con.sql("""COMMIT;""")

In [9]:
con.sql("""
SELECT * FROM Drug WHERE drug_cui = 'C0046056'
""")

┌──────────┬───────────────────────┐
│ drug_cui │         name          │
│ varchar  │        varchar        │
├──────────┼───────────────────────┤
│ C0046056 │ fludeoxyglucose (18F) │
└──────────┴───────────────────────┘

In [10]:
con.sql("""BEGIN TRANSACTION;""")

for line in open("drugs/disease_4_embedding.json", "r"):
  value = json.loads(line)

  query = f"""INSERT INTO Disorder VALUES ('{value["cui"]}', '{value["name"].replace("'", "''")}', '{value["definition"].replace("'", "''")}', {value["definitionEmbedding"]});"""
  #print(query)
  con.sql(query)

con.sql("""COMMIT;""")

In [11]:
con.sql("""BEGIN TRANSACTION;""")

df = pd.read_csv("drugs/moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():

  query = f"""INSERT INTO MOA VALUES ('{row["id"]}', '{row["name"].replace("'", "''")}');"""
  #print(query)
  con.sql(query)

con.sql("""COMMIT;""")

In [12]:
con.sql("""BEGIN TRANSACTION;""")
df = pd.read_csv("drugs/cui_indication_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
  query = f"""INSERT INTO DrugDisorder VALUES ('{row["from"]}', '{row["to"]}');"""
  con.sql(query)


con.sql("""COMMIT;""")

In [13]:
con.sql("""BEGIN TRANSACTION;""")
df = pd.read_csv("drugs/cui_moa_3.tsv", sep='\t', lineterminator='\n')

for index, row in df.iterrows():
  query = f"""INSERT INTO DrugMOA VALUES ('{row["from"]}', '{row["to"]}');"""
  con.sql(query)


con.sql("""COMMIT;""")

In [14]:
con.sql("""
FROM GRAPH_TABLE (drug_graph
  MATCH
  (i:Drug)-[m:MAY_TREAT]->(c:Disorder WHERE c.name = 'Alzheimer''s Disease')
  COLUMNS (i.name AS drug_name)
)
LIMIT 5;
""")

┌───────────────┐
│   drug_name   │
│    varchar    │
├───────────────┤
│ aducanumab    │
│ galantamine   │
│ valproic acid │
│ tacrine       │
│ donepezil     │
└───────────────┘

In [15]:
con.sql("""FROM GRAPH_TABLE (drug_graph
        MATCH
        (d:Drug WHERE d.name = 'medrysone')-[h:HAS_MOA]->(m:MOA)
        COLUMNS (m.name AS moa_name)
      )
      LIMIT 5;""")

┌──────────────────────────────────────────┐
│                 moa_name                 │
│                 varchar                  │
├──────────────────────────────────────────┤
│ Lipoxygenase Inhibitors                  │
│ Corticosteroid Hormone Receptor Agonists │
│ Glucocorticoid Receptor Agonists         │
└──────────────────────────────────────────┘

In [16]:
con.sql("""
SELECT Trials.PostingID as trial_id, Trials.drug_cui, drug_for_disease.drug_cui
FROM Trials,
    GRAPH_TABLE(
            drug_graph
                MATCH
                (i:Drug)-[m:MAY_TREAT]->(c:Disorder WHERE c.name = 'Alzheimer''s Disease')
                COLUMNS (i.drug_cui AS drug_cui)
    )  drug_for_disease
WHERE list_contains(Trials.drug_cui, drug_for_disease.drug_cui)
""")

┌──────────┬────────────┬────────────┐
│ trial_id │  drug_cui  │ drug_cui_1 │
│  int32   │ varchar[]  │  varchar   │
├──────────┼────────────┼────────────┤
│    19970 │ [C0527316] │ C0527316   │
│    19971 │ [C0527316] │ C0527316   │
│    19972 │ [C0527316] │ C0527316   │
│    19973 │ [C0527316] │ C0527316   │
│    19974 │ [C0527316] │ C0527316   │
│    19975 │ [C0527316] │ C0527316   │
│    19976 │ [C0527316] │ C0527316   │
│    19977 │ [C0527316] │ C0527316   │
│    19978 │ [C0527316] │ C0527316   │
│    19979 │ [C0527316] │ C0527316   │
│    19980 │ [C0527316] │ C0527316   │
│    20782 │ [C0527316] │ C0527316   │
│    20783 │ [C0527316] │ C0527316   │
│    20784 │ [C0527316] │ C0527316   │
│     4086 │ [C0649350] │ C0649350   │
│     4179 │ [C0649350] │ C0649350   │
│     4180 │ [C0649350] │ C0649350   │
│     4556 │ [C0649350] │ C0649350   │
│     4688 │ [C0649350] │ C0649350   │
│     4689 │ [C0649350] │ C0649350   │
│     4749 │ [C0649350] │ C0649350   │
│    21103 │ [C0649350] │

In [17]:
con.sql("""
SELECT Trials.PostingID as trial_id, Trials.drug_cui, Drug.drug_cui
FROM Trials, Drug, Disorder, DrugDisorder
WHERE Disorder.name = 'Alzheimer''s Disease' 
  AND Disorder.disorder_cui = DrugDisorder.disorder_cui 
  AND Drug.drug_cui = DrugDisorder.drug_cui 
  AND list_contains(Trials.drug_cui, Drug.drug_cui)
""")

┌──────────┬────────────┬──────────┐
│ trial_id │  drug_cui  │ drug_cui │
│  int32   │ varchar[]  │ varchar  │
├──────────┼────────────┼──────────┤
│    19970 │ [C0527316] │ C0527316 │
│    19971 │ [C0527316] │ C0527316 │
│    19972 │ [C0527316] │ C0527316 │
│    19973 │ [C0527316] │ C0527316 │
│    19974 │ [C0527316] │ C0527316 │
│    19975 │ [C0527316] │ C0527316 │
│    19976 │ [C0527316] │ C0527316 │
│    19977 │ [C0527316] │ C0527316 │
│    19978 │ [C0527316] │ C0527316 │
│    19979 │ [C0527316] │ C0527316 │
│    19980 │ [C0527316] │ C0527316 │
│    20782 │ [C0527316] │ C0527316 │
│    20783 │ [C0527316] │ C0527316 │
│    20784 │ [C0527316] │ C0527316 │
│     4086 │ [C0649350] │ C0649350 │
│     4179 │ [C0649350] │ C0649350 │
│     4180 │ [C0649350] │ C0649350 │
│     4556 │ [C0649350] │ C0649350 │
│     4688 │ [C0649350] │ C0649350 │
│     4689 │ [C0649350] │ C0649350 │
│     4749 │ [C0649350] │ C0649350 │
│    21103 │ [C0649350] │ C0649350 │
├──────────┴────────────┴──────────┤
│

In [18]:
con.sql("""
SELECT Trials.PostingID as trial_id, Trials.drug_cui, Drug.drug_cui
FROM Trials, Drug
JOIN DrugDisorder dd ON Drug.drug_cui = dd.drug_cui
JOIN Disorder d ON dd.disorder_cui = d.disorder_cui
WHERE d.name = 'Alzheimer''s Disease' AND list_contains(Trials.drug_cui, Drug.drug_cui)
""")

┌──────────┬────────────┬──────────┐
│ trial_id │  drug_cui  │ drug_cui │
│  int32   │ varchar[]  │ varchar  │
├──────────┼────────────┼──────────┤
│    19970 │ [C0527316] │ C0527316 │
│    19971 │ [C0527316] │ C0527316 │
│    19972 │ [C0527316] │ C0527316 │
│    19973 │ [C0527316] │ C0527316 │
│    19974 │ [C0527316] │ C0527316 │
│    19975 │ [C0527316] │ C0527316 │
│    19976 │ [C0527316] │ C0527316 │
│    19977 │ [C0527316] │ C0527316 │
│    19978 │ [C0527316] │ C0527316 │
│    19979 │ [C0527316] │ C0527316 │
│    19980 │ [C0527316] │ C0527316 │
│    20782 │ [C0527316] │ C0527316 │
│    20783 │ [C0527316] │ C0527316 │
│    20784 │ [C0527316] │ C0527316 │
│     4086 │ [C0649350] │ C0649350 │
│     4179 │ [C0649350] │ C0649350 │
│     4180 │ [C0649350] │ C0649350 │
│     4556 │ [C0649350] │ C0649350 │
│     4688 │ [C0649350] │ C0649350 │
│     4689 │ [C0649350] │ C0649350 │
│     4749 │ [C0649350] │ C0649350 │
│    21103 │ [C0649350] │ C0649350 │
├──────────┴────────────┴──────────┤
│

In [19]:
question = "joint-related disease"

question_embedding = get_embedding(question)

con.sql(f"""
SELECT name, definition
FROM Disorder
ORDER BY array_distance(definitionEmbedding, {question_embedding}::FLOAT[1536])
LIMIT 3;
""")



┌──────────────────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┐
│         name         │                                                                                                                                                                                              definition                                                                                                                                                                                               │
│       varchar        │                                                                                                                                              

In [20]:
con.sql(f"""
SELECT Trials.PostingID AS trial_id, target_disease.name, drug_for_disorder.drug_name FROM 
  Trials, (
  SELECT disorder_cui, name
    FROM Disorder
    ORDER BY array_distance(definitionEmbedding, {question_embedding}::FLOAT[1536])
    LIMIT 3
  ) target_disease,
  GRAPH_TABLE(
    drug_graph
      MATCH (i:Drug)-[m:MAY_TREAT]->(c:Disorder)
      COLUMNS (i.drug_cui AS drug_cui, i.name AS drug_name,c.disorder_cui AS disorder_cui)
    )  drug_for_disorder
WHERE target_disease.disorder_cui = drug_for_disorder.disorder_cui AND list_contains(Trials.drug_cui, drug_for_disorder.drug_cui)
""")

┌──────────┬──────────────────────┬──────────────────────┐
│ trial_id │         name         │      drug_name       │
│  int32   │       varchar        │       varchar        │
├──────────┼──────────────────────┼──────────────────────┤
│     4829 │ Rheumatoid Arthritis │ certolizumab pegol   │
│     3610 │ Rheumatoid Arthritis │ nabumetone           │
│      121 │ Rheumatoid Arthritis │ naproxen             │
│      122 │ Rheumatoid Arthritis │ naproxen             │
│      123 │ Rheumatoid Arthritis │ naproxen             │
│     3474 │ Rheumatoid Arthritis │ naproxen             │
│     3475 │ Rheumatoid Arthritis │ naproxen             │
│     3476 │ Rheumatoid Arthritis │ naproxen             │
│     3477 │ Rheumatoid Arthritis │ naproxen             │
│     3478 │ Rheumatoid Arthritis │ naproxen             │
│       ·  │          ·           │    ·                 │
│       ·  │          ·           │    ·                 │
│       ·  │          ·           │    ·                

In [21]:
con.sql("""
                    FROM GRAPH_TABLE (drug_graph
                    MATCH
                    (mo:MOA)<-[h:HAS_MOA]-(i:Drug)-[m:MAY_TREAT]->(c:Disorder WHERE c.name = 'Alzheimer''s Disease')
                    COLUMNS (mo.name AS moa_name)
                    )
                    LIMIT 5;
""")

┌─────────────────────────────────────────────┐
│                  moa_name                   │
│                   varchar                   │
├─────────────────────────────────────────────┤
│ Amyloid Beta-directed Antibody Interactions │
│ Cholinesterase Inhibitors                   │
│ GABA B Receptor Interactions                │
│ Cholinesterase Inhibitors                   │
│ Cholinesterase Inhibitors                   │
└─────────────────────────────────────────────┘

In [22]:
con.sql("""
                    FROM GRAPH_TABLE (drug_graph
                    MATCH
                    (mo:MOA WHERE mo.name = 'GABA B Receptor Interactions')<-[h:HAS_MOA]-(i:Drug)-[m:MAY_TREAT]->(c:Disorder)
                    COLUMNS (c.name AS disorder_name)
                    )
                    LIMIT 5;
""")

┌─────────────────────┐
│    disorder_name    │
│       varchar       │
├─────────────────────┤
│ Alzheimer's Disease │
│ Epilepsy            │
│ Absence Epilepsy    │
│ Mood Disorders      │
│ Bipolar Disorder    │
└─────────────────────┘

In [23]:
con.sql("""
    SELECT Trials.StudyTitle as StudyTitle, drug_for_disease.drug_cui
    FROM Trials,
    GRAPH_TABLE(
        drug_graph
        MATCH
        (i:Drug)-[m:MAY_TREAT]->(c:Disorder WHERE c.name = 'Alzheimer''s Disease')
        COLUMNS (i.drug_cui AS drug_cui)
    )  drug_for_disease
    WHERE list_contains(Trials.drug_cui, drug_for_disease.drug_cui)
    LIMIT 5;
""")

┌───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬──────────┐
│                                                                                        StudyTitle                                                                                         │ drug_cui │
│                                                                                          varchar                                                                                          │ varchar  │
├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┼──────────┤
│ Clinical efficacy and safety of donepezil on cognitive and global function in patients with Alzheimer's disease. A 24-week, multicenter, double-blind, placebo-controlled study in Japan. │ C05273

In [24]:
con.sql("""
SELECT Sponsor, Phase, COUNT(PostingID) AS trial_count
                FROM Trials, Drug
                WHERE LOWER(Drug.name) = LOWER('Fluticasone pRopionate') AND list_contains(Trials.drug_cui, Drug.drug_cui)
                GROUP BY Sponsor, Phase
                ORDER BY Sponsor, Phase;
""")

┌──────────┬─────────┬─────────────┐
│ Sponsor  │  Phase  │ trial_count │
│ varchar  │ varchar │    int64    │
├──────────┼─────────┼─────────────┤
│ GSK      │ Phase 1 │          27 │
│ GSK      │ Phase 2 │           6 │
│ GSK      │ Phase 3 │          39 │
│ GSK      │ Phase 4 │          80 │
│ GSK      │ nan     │           3 │
│ Novartis │ Phase 2 │           1 │
└──────────┴─────────┴─────────────┘

In [25]:
con.sql("""
SELECT Disorder.name 
                        FROM DrugDisorder, Drug, Disorder
                        WHERE DrugDisorder.drug_cui = Drug.drug_cui
                        AND DrugDisorder.disorder_cui = Disorder.disorder_cui
                        AND LOWER(Drug.name) = LOWER('fluocinolone acetonide');
""")

┌────────────────────┐
│        name        │
│      varchar       │
├────────────────────┤
│ Inflammation       │
│ Hand Dermatoses    │
│ Foot Dermatoses    │
│ Dermatitis, Atopic │
│ Scalp Dermatoses   │
│ Pruritus           │
│ Psoriasis          │
│ Facial Dermatoses  │
│ Leg Dermatoses     │
└────────────────────┘

In [26]:
con.close()